In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
dataset=pd.read_csv("dataset.csv")
print(dataset.shape)
data_size=len(dataset)
print (data_size)

(16602, 3)
16602


In [127]:
import re
import nltk
from urllib.parse import urlparse
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus=[]

for i in range(0,data_size):
    tweet=dataset['text'][i]
    
    tweet = re.sub('[^a-zA-Z]', ' ', dataset['text'][i])
    tweet=tweet.lower()
    tweet=tweet.split()
    ps=PorterStemmer()
    tweet=[ps.stem(word) for word in tweet if word not in set(stopwords.words('english')) and len(word)>2]
    tweet=' '.join(tweet)
    corpus.append(tweet)
    
    

In [128]:
corpus_temp=corpus

In [129]:
corpus=corpus_temp

In [130]:
words=[]
for i in corpus:
    for word in i:
        words.append(i)
print (len(words))
print ((corpus[0]))

1050357
nepal nsent whatsapp http cla kaa


In [131]:
import collections
counter=collections.Counter(words)
common=counter.most_common(50)
common_words=[]
for i in range(0,50):
    wo=common[i][0].split()
    for w in wo:
        common_words.append(w)

In [132]:
print (len(common_words))
common_words=list(set(common_words))
print (len(common_words))
print (common_words)

585
358
['right', 'odisha', 'held', 'satguru', 'gandhi', 'post', 'growth', 'blood', 'forward', 'neetuupadhyay', 'give', 'rajasthan', 'understandin', 'ccsqoqbz', 'cow', 'pondychrri', 'msghelpearthquakevictim', 'east', 'disgrac', 'poin', 'sadhvi', 'sikhism', 'iamsrk', 'visit', 'pl', 'offer', 'non', 'fallen', 'rajya', 'love', 'countri', 'warn', 'nearthquak', 'selfless', 'complement', 'suicid', 'fund', 'text', 'break', 'indiatoday', 'bless', 'donat', 'ilariaurbinati', 'come', 'moron', 'thi', 'help', 'sinceer', 'friendli', 'bike', 'gurdeepinsan', 'sakshi', 'msghe', 'meal', 'relief', 'worst', 'allow', 'apjabdulkalam', 'wing', 'golden', 'ministri', 'across', 'indiawithnep', 'manishtamak', 'govt', 'volunt', 'remind', 'mimarathinew', 'ipckf', 'rememb', 'indianexpress', 'amp', 'namo', 'medianorm', 'word', 'heart', 'pac', 'bihar', 'aasra', 'expect', 'chairman', 'anxieti', 'derasachasauda', 'highest', 'magic', 'miss', 'hit', 'pwrful', 'media', 'madhumarasini', 'hcz', 'murder', 'yard', 'servic', 'l

In [133]:
imp_words=['medic','need','give','relief','fund','food','donat','aid','water','meal','send','offer','financ','blood']


In [134]:
common_words=[i for i in common_words if i not in imp_words]
print (len(common_words))

345


In [137]:
final_corpus=[]
for i in corpus:
    final_corpus.append(' '.join([w for w in i.split() if w not in common_words]))

In [138]:
words=[]
for i in final_corpus:
    for word in i:
        words.append(i)
print (len(words))

631201


In [139]:
Y=dataset.iloc[:,2].values
Y=list(Y)
print (len(final_corpus))
seen = set()
result = []
removed_indices=[]
for i in range(0,data_size):
    if final_corpus[i] not in seen:
        seen.add(final_corpus[i])
        result.append(final_corpus[i])
    else:
        removed_indices.insert(0,i)
final_corpus=result
for i in removed_indices:
    del(Y[i])
print (len(final_corpus))
print (len(Y))

16602
14252
14252


In [140]:
count=0
for i in Y:
    if i==2 or i==1:
        count+=1
print (count)


875


In [141]:
delIndex=[]
l=len(Y)
c=0
for i in range(0,l):
    if(Y[i]==0):
        if(c==count):
            delIndex=[i]+delIndex
        else:
            c+=1
for i in delIndex:
    final_corpus.pop(i)
    Y.pop(i)
print(c)

875


In [142]:
print(len(final_corpus))
print(len(Y))

1750
1750


In [143]:
from sklearn.cross_validation import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(final_corpus,Y,test_size=0.20,random_state=0)


In [149]:
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn import feature_selection
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import tree
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn import svm

LinearSVC1 = LinearSVC(tol=1e-4,  C = 0.10000000000000001)
f5 = feature_selection.RFE(estimator=LinearSVC1, n_features_to_select=500, step=1)

pipeline=Pipeline([
    #('vectorizer',  CountVectorizer()),
    ('features', FeatureUnion([
       ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features= 4000))])), 
    ('rfe_feature_selection', f5),
    ('classifier', LinearSVC1),
    ])
    #('tfidf_transformer',  TfidfTransformer()),
    #('classifier',  svm.SVC())
    #('classifier',  tree.DecisionTreeClassifier()) 
    #])
pipeline.fit(X_train,Y_train)
y_pred=pipeline.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
print (cm)

[[169   4   8]
 [ 13  16   6]
 [ 23   2 109]]


In [150]:
from sklearn.metrics import accuracy_score
print (accuracy_score(Y_test,y_pred))

0.84


In [14]:
support=pipeline.named_steps['rfe_feature_selection'].support_
feature_names = pipeline.named_steps['features'].get_feature_names()

In [15]:
import graphviz
import pydotplus
from sklearn import tree
from sklearn.externals.six import StringIO
dot_data = StringIO()
tree.export_graphviz(pipeline.named_steps['classifier'],out_file=dot_data,feature_names=feature_names,class_names=['0','1','2'],filled=True, rounded=True,special_characters=True) 
pydotplus.graph_from_dot_data(dot_data.getvalue()).write_png("tree.png")

True